In [12]:
from dataclasses import dataclass

from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
class Education(BaseModel):
    academic_degrees: list[str] = Field(description="List of relevant academic degrees")
    accreditations: list[str] = Field(
        default_factory=list, description="List of accreditations from PEC or recognized bodies"
    )
    weightage: float = Field(default=0.15, description="Weightage percentage between 10-15%")


class ProfessionalExperience(BaseModel):
    total_years: int = Field(description="Total years of experience in relevant field")
    project_experience: list[str] = Field(description="Specific project-related experience matching project scope")
    leadership_roles: list[str] = Field(description="Experience in leadership positions")
    weightage: float = Field(default=0.3, description="Weightage percentage between 25-30%")


class PECRegistration(BaseModel):
    registration_type: str = Field(description="Type of PEC registration (PE/RE)")
    certifications: list[str] = Field(description="Additional professional certifications")
    weightage: float = Field(default=0.1, description="Weightage percentage between 5-10%")


class ProjectExperience(BaseModel):
    similar_projects: list[str] = Field(description="Experience in similar projects")
    completed_projects: int = Field(description="Number of completed projects")
    project_scale: list[str] = Field(description="Scale of completed projects")
    weightage: float = Field(default=0.25, description="Weightage percentage between 20-25%")


class Training(BaseModel):
    professional_training: list[str] = Field(description="Professional training and workshops")
    cpd_points: int = Field(description="CPD points as per PEC requirements")
    weightage: float = Field(default=0.05, description="Weightage percentage between 2.5-5%")


class Publications(BaseModel):
    publications: list[str] = Field(description="Relevant publications and technical papers")
    research_work: list[str] = Field(description="Research work in relevant field")
    weightage: float = Field(default=0.05, description="Weightage percentage between 2.5-5%")


class CVEvaluationCriteria(BaseModel):
    education: Education
    professional_experience: ProfessionalExperience
    pec_registration: PECRegistration
    project_experience: ProjectExperience
    training: Training
    publications: Publications | None = None


@dataclass
class RFP:
    datasheet: str
    terms_of_reference: str

    def __str__(self) -> str:
        return (
            f"\n<RFP>\n\n"
            f"<datasheet>\n{self.datasheet}\n</datasheet>\n\n"
            f"<terms_of_reference>\n{self.terms_of_reference}\n</terms_of_reference>\n\n"
            f"</RFP>"
        )


In [ ]:
criteria_agent = Agent(
    "openai:gpt-4o-mini",
    result_type=CVEvaluationCriteria,
    system_prompt=(
        "You are an expert in evaluating engineering CVs and RFPs. Your task is to analyze the RFP's datasheet "
        "and terms of reference to extract specific evaluation criteria for CVs based on the company's requirements.\n\n"
        "For each category, carefully review the datasheet and terms of reference to:\n"
        "1. Identify required qualifications, experience, and credentials explicitly mentioned\n"
        "2. Determine appropriate weightage within the specified ranges based on emphasis in the documents\n"
        "3. List specific requirements that align with the project scope and deliverables\n\n"
        "Pay special attention to:\n"
        "- Minimum qualification requirements in the datasheet\n"
        "- Years of experience specified in the terms of reference\n"
        "- Specific technical expertise required for deliverables\n"
        "- Any mandatory certifications or registrations\n\n"
        "Ensure all weightages sum to 100% and stay within their specified ranges:\n"
        "- Education & Qualification: 10-15%\n"
        "- Professional Experience: 25-30%\n"
        "- PEC Registration & Certifications: 5-10%\n"
        "- Relevant Project Experience: 20-25%\n"
        "- Training & CPD: 2.5-5%\n"
        "- Publications & Research: 2.5-5%\n\n"
        "Return a structured response matching the required format."
    ),
    deps_type=RFP,
)


@criteria_agent.system_prompt(dynamic=True)
def system_prompt(ctx: RunContext[RFP]) -> str:
    return str(ctx.deps)
